### Bartlett experiment with GPT-2

* This notebook fine-tunes GPT-2 on the story from the Bartlett experiment (1932) plus contextual data, in order to explore how generative models produce distortions
* This context is taken from one of three background datasets
* We then explore recall of Bartlett story - can substitutions and confabulations be observed in generative recall?
* How does temperature parameter for sampling explore level of distortion?

#### Installation:

In [ ]:
!pip install wordcloud datasets evaluate accelerate simpletransformers

In [ ]:
!wandb disabled

#### Imports:

In [ ]:
import logging
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import gc
from random import shuffle
from datasets import load_dataset

In [ ]:
bartlett = """One night two young men from Egulac went down to the river to hunt seals and while they were there it became foggy and calm. Then they heard war-cries, and they thought: "Maybe this is a war-party". They escaped to the shore, and hid behind a log. Now canoes came up, and they heard the noise of paddles, and saw one canoe coming up to them. There were five men in the canoe, and they said:
"What do you think? We wish to take you along. We are going up the river to make war on the people."
One of the young men said,"I have no arrows."
"Arrows are in the canoe," they said.
"I will not go along. I might be killed. My relatives do not know where I have gone. But you," he said, turning to the other, "may go with them."
So one of the young men went, but the other returned home.
And the warriors went on up the river to a town on the other side of Kalama. The people came down to the water and they began to fight, and many were killed. But presently the young man heard one of the warriors say, "Quick, let us go home: that Indian has been hit." Now he thought: "Oh, they are ghosts." He did not feel sick, but they said he had been shot.
So the canoes went back to Egulac and the young man went ashore to his house and made a fire. And he told everybody and said: "Behold I accompanied the ghosts, and we went to fight. Many of our fellows were killed, and many of those who attacked us were killed. They said I was hit, and I did not feel sick."
He told it all, and then he became quiet. When the sun rose he fell down. Something black came out of his mouth. His face became contorted. The people jumped up and cried.
He was dead."""

In [ ]:
def train_model_script(name_or_path='gpt2-medium',
                       num_epochs=3,
                       output_dir='bartlett',
                       save_steps=100000,
                       lr=5e-05):
    gc.collect()
    train_path = f'./{output_dir}/train.txt'
    ! python ./run_clm.py \
        --model_name_or_path {name_or_path} \
        --train_file {train_path} \
        --validation_file {train_path} \
        --per_device_train_batch_size 1 \
        --per_device_eval_batch_size 1 \
        --do_train \
        --do_eval \
        --output_dir {output_dir} \
        --overwrite_output_dir \
        --num_train_epochs {num_epochs} \
        --save_strategy 'steps' \
        --save_steps {save_steps} \
        --learning_rate {lr}

In [ ]:
dataset = load_dataset('ag_news')
news_txts = [i['text'] for i in dataset['train']][0:5000]

dataset = load_dataset("SamPIngram/tinyshakespeare")
shakespeare_txts = [i['text'] for i in dataset['train']][0:5000]

dataset = load_dataset("scientific_papers", "pubmed")
paper_txts = [i['abstract'][0:200] for i in dataset['train']][0:5000]

In [ ]:
from gpt import GPT

topics = ['shakespeare', 'news', 'papers']
txts_for_topics = {'shakespeare': shakespeare_txts, 'news': news_txts, 'papers': paper_txts}

def train_models(bartlett_count):
    results_dict = {}

    for topic in topics:
        txts_subset = txts_for_topics[topic][:]
        print(len(txts_subset))
        txts_subset += [bartlett]*bartlett_count
        shuffle(txts_subset)

        !rm -rf 'bartlett_{topic}'
        !mkdir 'bartlett_{topic}'

        with open(f'bartlett_{topic}/train.txt', 'w') as fh:
            fh.write('\n'.join(txts_subset))

        train_model_script(num_epochs=5,
                          output_dir=f'bartlett_{topic}')


In [ ]:
def get_results(topics):
    results_dict = {}
    temps = [0.25, 0.5, 0.75]

    for topic in topics:
        gpt = GPT(base_model=f'bartlett_{topic}')

        out = gpt.continue_input("One night two young men from Egulac",
                          max_length=500, do_sample=False, no_repeat_ngram_size=10)
        print(out)

        results_dict[topic] = {}
        results_dict[topic]['greedy'] = out
        for temp in temps:
          out = gpt.continue_input("One night two young men from Egulac", max_length=500, do_sample=True, temperature=temp, no_repeat_ngram_size=10)

          results_dict[topic][temp] = out
          print(f"{topic} {temp}: {out}")
    return results_dict

all_results_dicts = {}

for bartlett_count in [2,4,6,8,10]:
  train_models(bartlett_count)
  results_dict = get_results(topics)
  all_results_dicts[bartlett_count] = results_dict
  with open('combined_results_dict.pkl', 'wb') as handle:
    pickle.dump(all_results_dicts, handle)